<a href="https://colab.research.google.com/github/Shivani8780/Matrimonial-Profile/blob/main/matching_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import re

# Detect encoding manually if needed (e.g., 'ISO-8859-1' or 'cp1252')
file_path = 'data_for_matching.csv'

try:
    # Attempt to load the file with explicit encoding
    df = pd.read_csv(file_path, encoding='ISO-8859-1')  # Replace 'ISO-8859-1' with the correct encoding if necessary
except UnicodeDecodeError as e:
    print(f"UnicodeDecodeError: {e}")
    print("Attempting to load the file with a different encoding.")
    df = pd.read_csv(file_path, encoding='utf-8', errors='replace')  # Replace problematic characters

# Strip leading and trailing spaces from column names for easier handling
df.columns = [col.strip() for col in df.columns]

# Display the cleaned column names and the first few rows to confirm changes
print(df.columns)
print(df.head())

# Function to clean height values
def clean_height(height):
    if pd.isnull(height):
        return None
    height = str(height).lower()
    if 'feet' in height or 'ft' in height:
        feet = re.findall(r'(\d+)\s*feet', height)
        inches = re.findall(r'(\d+)\s*inch', height)
        feet = int(feet[0]) if feet else 0
        inches = int(inches[0]) if inches else 0
        return feet * 12 + inches
    elif '\'' in height:
        parts = height.split('\'')
        feet = int(parts[0]) if parts[0].isdigit() else 0
        inches = int(parts[1].replace('"', '')) if len(parts) > 1 and parts[1].replace('"', '').isdigit() else 0
        return feet * 12 + inches
    else:
        try:
            return float(height)
        except ValueError:
            return None

# Apply the height cleaning function
if 'Height (In Feet)' in df.columns:
    df['Height (In Feet)'] = df['Height (In Feet)'].apply(clean_height)

# Function to clean weight values
def clean_weight(weight):
    if pd.isnull(weight):
        return None
    weight = re.sub(r'[^\d.]', '', str(weight))  # Remove non-numeric characters except for one decimal point
    weight = re.sub(r'\.+', '.', weight)  # Remove extra decimal points
    if weight.endswith('.'):
        weight = weight[:-1]
    try:
        return float(weight) if weight else None
    except ValueError:
        return None

# Apply the weight cleaning function
if 'Weight (In KG)' in df.columns:
    df['Weight (In KG)'] = df['Weight (In KG)'].apply(clean_weight)

# Clean text fields
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Save the cleaned data
output_file = 'cleaned_data_for_matching.csv'
df.to_csv(output_file, index=False, encoding='utf-8')
print(f"Data cleaning complete. Cleaned data saved to '{output_file}'.")


Index(['Name Of Candidate', 'Candidate Gender',
       'Who is doing this Registration ?', 'MOBILE_NO', 'CITY_COUNTRY',
       'Candidate Current Residence Area Category',
       'Candidate Current Country', 'Visa or Residence Status Of Candidate',
       'Date Of Birth', 'Time Of Birth', 'Birth Place', 'Height (In Feet)',
       'Weight (In KG)', 'Highest Education Category', 'Education Details',
       'Job/Business/Occupation Category',
       'Details on Job / Business / Occupation',
       'Mention your Salary or Income (optional)', 'Marriage Status', 'Nadi',
       'Shani / Managal', 'Type Of Brahmin', 'Gotra',
       'Candidate has Any Mental / Physical Disability or Minor Problem',
       'Siblings Details (Brother/Sister) & Contact Number',
       'Choice Of Education', 'Choice Of Location',
       'Choice Of Age Gap/ Difference in Year', 'Any Other Choice'],
      dtype='object')
         Name Of Candidate Candidate Gender Who is doing this Registration ?  \
0                

<ipython-input-2-990b02abe903>:66: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [3]:
# Function to clean date formats
def clean_date(date):
    try:
        return pd.to_datetime(date, errors='coerce').strftime('%d-%m-%Y')
    except:
        return None

# Apply the date cleaning function
df['Date Of Birth'] = df['Date Of Birth'].apply(clean_date)
# Clean text fields
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

df['Candidate has Any Mental / Physical Disability or Minor Problem'] = df['Candidate has Any Mental / Physical Disability or Minor Problem'].str.lower().str.strip()

<ipython-input-3-1e91771a864e>:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(date, errors='coerce').strftime('%d-%m-%Y')
<ipython-input-3-1e91771a864e>:4: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(date, errors='coerce').strftime('%d-%m-%Y')
<ipython-input-3-1e91771a864e>:4: UserWarning: Parsing dates in %d %m %Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(date, errors='coerce').strftime('%d-%m-%Y')
<ipython-input-3-1e91771a864e>:4: UserWarning: Parsing dates in %d/ %m/ %Y  . format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd

In [4]:
df['Choice Of Education'] = df['Choice Of Education'].str.lower().str.strip()

In [5]:
def clean_salary(salary):
    if pd.isnull(salary):
        return None
    salary = str(salary).lower()
    # Extract numeric values
    numbers = re.findall(r'\d+', salary)
    if 'lac' in salary:
        return int(numbers[0]) * 100000 if numbers else None
    elif 'k' in salary:
        return int(numbers[0]) * 1000 if numbers else None
    elif 'cad' in salary:
        return int(numbers[0]) * 60 if numbers else None  # Example conversion rate
    else:
        return int(numbers[0]) if numbers else None

df['Mention your Salary or Income (optional)'] = df['Mention your Salary or Income (optional)'].apply(clean_salary)

In [6]:
df['Marriage Status'] = df['Marriage Status'].str.lower().str.strip()
df['Education Details'] = df['Education Details'].str.lower().str.strip()

In [7]:
df.fillna('Unknown', inplace=True)

<ipython-input-7-876f1fdd0c9d>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('Unknown', inplace=True)


In [8]:
df['Candidate has Any Mental / Physical Disability or Minor Problem'] = df['Candidate has Any Mental / Physical Disability or Minor Problem'].str.lower().str.strip()
df['Choice Of Education'] = df['Choice Of Education'].str.lower().str.strip()
df['Marriage Status'] = df['Marriage Status'].str.lower().str.strip()
df['Shani / Managal'] = df['Shani / Managal'].str.lower().str.strip()

In [9]:
df['Gotra'] = df['Gotra'].str.lower().str.strip()


In [11]:
import pandas as pd
import numpy as np
import re

# Step 1: Data Loading
def load_education_data(file_path):
    """
    Load education data from various file types with encoding handling.
    """
    try:
        # Support multiple file types
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path, encoding='utf-8', errors='replace')  # Replace invalid characters
        elif file_path.endswith(('.xlsx', '.xls')):
            df = pd.read_excel(file_path)
        elif file_path.endswith('.txt'):
            df = pd.read_csv(file_path, sep='\t', encoding='utf-8', errors='replace')  # Replace invalid characters
        else:
            raise ValueError("Unsupported file type")

        # Strip column names to remove leading/trailing spaces
        df.columns = df.columns.str.strip()

        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        return pd.DataFrame()

# Step 2: Comprehensive Preprocessing
class EducationPreprocessor:
    def __init__(self, df):
        self.df = df.copy()

    def clean_text(self):
        """
        Clean and standardize text
        """
        # Convert to string and handle NaN
        self.df['Education Details'] = self.df['Education Details'].astype(str)

        # Convert to title case
        self.df['Education Details'] = self.df['Education Details'].str.title()

        # Remove extra whitespaces
        self.df['Education Details'] = self.df['Education Details'].str.strip()

        return self

    def standardize_degrees(self):
        """
        Standardize degree abbreviations and names
        """
        # Dictionary of abbreviations and their full forms
        degree_mapping = {
            r'\bB\.?Com\.?': 'Bachelor of Commerce',
            r'\bB\.?E\.?': 'Bachelor of Engineering',
            r'\bB\.?Tech\.?': 'Bachelor of Technology',
            r'\bM\.?Sc\.?': 'Master of Science',
            r'\bM\.?Com\.?': 'Master of Commerce',
            r'\bM\.?B\.?A\.?': 'Master of Business Administration',
            r'\bM\.?Tech\.?': 'Master of Technology',
            r'\bPh\.?D\.?': 'Doctor of Philosophy',
            r'\bB\.?A\.?': 'Bachelor of Arts',
            r'\bB\.?Sc\.?': 'Bachelor of Science'
        }

        # Apply replacements
        for pattern, replacement in degree_mapping.items():
            self.df['Education Details'] = self.df['Education Details'].str.replace(
                pat=pattern,
                repl=replacement,
                regex=True
            )

        return self

    def categorize_education(self):
        """
        Categorize education levels
        """
        def get_education_category(education):
            # Define category rules
            categories = [
                (r'(Doctor of Philosophy|Ph\.?D)', 'Doctoral'),
                (r'(Master of|M\.|Post Graduate)', 'Postgraduate'),
                (r'(Bachelor of|B\.)', 'Undergraduate'),
                (r'(10th|12th|High School|Ssc|Hsc)', 'High School'),
                (r'(Diploma|Certificate)', 'Diploma')
            ]

            for pattern, category in categories:
                if re.search(pattern, str(education), re.IGNORECASE):
                    return category

            return 'Other'

        # Add education category column
        self.df['Education_Category'] = self.df['Education Details'].apply(get_education_category)

        return self

    def extract_specialization(self):
        """
        Extract specialization from education string
        """
        def get_specialization(education):
            # Patterns to extract specialization
            patterns = [
                r'in\s(.+)$',  # Matches "in Something"
                r'$$([^)]+)$$',  # Matches content within parentheses
            ]

            for pattern in patterns:
                match = re.search(pattern, str(education), re.IGNORECASE)
                if match:
                    return match.group(1).strip()

            return 'Not Specified'

        # Add specialization column
        self.df['Specialization'] = self.df['Education Details'].apply(get_specialization)

        return self

    def identify_international_education(self):
        """
        Identify international education
        """
        international_countries = [
            'Canada', 'Usa', 'Uk', 'Australia', 'Germany',
            'France', 'Netherlands', 'Singapore'
        ]

        def check_international_education(education):
            for country in international_countries:
                if country.lower() in str(education).lower():
                    return country
            return 'Domestic'

        # Add international education column
        self.df['International_Education'] = self.df['Education Details'].apply(check_international_education)

        return self

    def generate_insights(self):
        """
        Generate insights from preprocessed data
        """
        insights = {
            'Total_Entries': len(self.df),
            'Education_Category_Distribution': self.df['Education_Category'].value_counts(),
            'Top_Specializations': self.df['Specialization'].value_counts().head(10),
            'International_Education_Distribution': self.df['International_Education'].value_counts()
        }

        return insights

    def preprocess(self):
        """
        Execute all preprocessing steps
        """
        return (self
                .clean_text()
                .standardize_degrees()
                .categorize_education()
                .extract_specialization()
                .identify_international_education())

# Main Execution
def main():
    # Load data
    file_path = 'data_for_matching.csv'  # Replace with your file path
    df = load_education_data(file_path)

    # Preprocess data
    preprocessor = EducationPreprocessor(df)
    processed_df = preprocessor.preprocess().df

    # Generate insights
    insights = preprocessor.generate_insights()

    # Print insights
    for key, value in insights.items():
        print(f"\n{key}:")
        print(value)

    # Optional: Save processed data
    processed_df.to_csv('processed_data.csv', index=False)

    return processed_df

# Run the preprocessing
if __name__ == '__main__':
    processed_dataframe = main()

Error loading data: read_csv() got an unexpected keyword argument 'errors'


KeyError: 'Education Details'

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

def preprocess_dob(dob_series):
    """
    Comprehensive date of birth preprocessing function

    Parameters:
    -----------
    dob_series : pandas Series
        Series containing date of birth values

    Returns:
    --------
    Preprocessed DataFrame with multiple date-related columns
    """

    def clean_and_convert_date(date_str):
        """
        Advanced date cleaning and conversion
        """
        if pd.isna(date_str):
            return pd.NaT

        # Convert to string and strip
        date_str = str(date_str).strip()

        # Remove extra spaces and special characters
        date_str = re.sub(r'[^\w\s,.-]', '', date_str)

        # Comprehensive date format handling
        date_formats = [
            '%d-%m-%Y',    # 11-09-1989
            '%d/%m/%Y',    # 29/06/1998
            '%d.%m.%Y',    # 21.07.84
            '%d %B %Y',    # 11th December, 1993
            '%B %d, %Y',   # April 02, 1995
            '%d-%b-%y',    # 30-Jul-97
            '%Y',          # 1991
            '%m/%d/%Y',    # 11/29/1995
            '%d %b %Y',    # 9th august 1994
            '%d-%B-%Y',    # 11-September-1989
            '%B %d %Y',    # September 11 1989
        ]

        # Try parsing with multiple formats
        for fmt in date_formats:
            try:
                parsed_date = pd.to_datetime(date_str, format=fmt)
                return parsed_date
            except:
                continue

        # Fallback parsing methods
        try:
            # Try general parsing
            parsed_date = pd.to_datetime(date_str, errors='coerce')
            return parsed_date
        except:
            return pd.NaT

    # Create DataFrame with robust conversion
    df = pd.DataFrame()

    # Ensure proper datetime conversion
    df['original_dob'] = dob_series

    # Apply advanced cleaning and conversion
    df['cleaned_dob'] = dob_series.apply(clean_and_convert_date)

    # Safely extract date components
    def safe_extract(series, attr):
        """
        Safely extract date components
        """
        try:
            return series.dt.__getattr__(attr)
        except:
            return pd.Series([np.nan] * len(series))

    # Extract date components with error handling
    df['year'] = safe_extract(df['cleaned_dob'], 'year')
    df['month'] = safe_extract(df['cleaned_dob'], 'month')
    df['day'] = safe_extract(df['cleaned_dob'], 'day')

    # Age calculation with error handling
    current_year = datetime.now().year
    df['age'] = df['year'].apply(lambda x: current_year - x if pd.notna(x) else np.nan)

    # Age group categorization
    def categorize_age(age):
        if pd.isna(age):
            return 'Unknown'
        elif age < 20:
            return 'Under 20'
        elif 20 <= age < 30:
            return '20-29'
        elif 30 <= age < 40:
            return '30-39'
        elif 40 <= age < 50:
            return '40-49'
        else:
            return '50+'

    df['age_group'] = df['age'].apply(categorize_age)

    # Validate dates with comprehensive checks
    def validate_date(row):
        try:
            # Check if date is not NaT
            if pd.isnull(row['cleaned_dob']):
                return False

            # Exclude future dates
            if row['cleaned_dob'] > pd.Timestamp.now():
                return False

            # Exclude extremely old dates
            if pd.notna(row['year']) and row['year'] < 1940:
                return False

            return True
        except:
            return False

    df['is_valid_date'] = df.apply(validate_date, axis=1)

    # Advanced date format detection
    def detect_date_format(date_str):
        if pd.isna(date_str):
            return 'Unknown'

        date_str = str(date_str)

        formats = [
            (r'^\d{2}-\d{2}-\d{4}$', 'DD-MM-YYYY'),
            (r'^\d{1,2}/\d{1,2}/\d{4}$', 'DD/MM/YYYY'),
            (r'^\d{1,2}\.\d{1,2}\.\d{2,4}$', 'DD.MM.YYYY'),
            (r'^\w+\s\d{1,2},?\s\d{4}$', 'Month DD, YYYY'),
            (r'^\d{1,2}-\w{3}-\d{2}$', 'DD-Mon-YY'),
            (r'^\d{4}$', 'YYYY'),
        ]

        for pattern, format_name in formats:
            if re.match(pattern, date_str):
                return format_name

        return 'Other'

    df['date_format'] = dob_series.apply(detect_date_format)

    return df

def main():
    # Example usage with error handling
    try:
        # Option 1: Direct list input
        dob_series = pd.Series([
            '11-09-1989', '21-10-1977', '16-07-1991',
            '30-Mar-97', 'June 13 1995', '27051998'
        ])

        # Preprocess dates
        preprocessed_df = preprocess_dob(dob_series)

        # Save the output to a CSV file
        output_file = 'preprocessed_dob.csv'
        preprocessed_df.to_csv(output_file, index=False)
        print(f"\nPreprocessed data saved to {output_file}")

        # Display results
        print("\nPreprocessed Date of Birth Data:")
        print(preprocessed_df)

        # Save the output to a CSV file
        output_file = 'preprocessed_dob.csv'
        preprocessed_df.to_csv(output_file, index=False)
        print(f"\nPreprocessed data saved to {output_file}")

        # Analysis
        print("\nAge Group Distribution:")
        print(preprocessed_df['age_group'].value_counts())

        print("\nDate Format Distribution:")
        print(preprocessed_df['date_format'].value_counts())

        print("\nValid Dates:")
        print(preprocessed_df['is_valid_date'].value_counts())

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


Preprocessed Date of Birth Data:
   original_dob cleaned_dob  year  month  day  age age_group  is_valid_date  \
0    11-09-1989  1989-09-11   NaN    NaN  NaN  NaN   Unknown           True   
1    21-10-1977  1977-10-21   NaN    NaN  NaN  NaN   Unknown           True   
2    16-07-1991  1991-07-16   NaN    NaN  NaN  NaN   Unknown           True   
3     30-Mar-97  1997-03-30   NaN    NaN  NaN  NaN   Unknown           True   
4  June 13 1995  1995-06-13   NaN    NaN  NaN  NaN   Unknown           True   
5      27051998         NaT   NaN    NaN  NaN  NaN   Unknown          False   

      date_format  
0      DD-MM-YYYY  
1      DD-MM-YYYY  
2      DD-MM-YYYY  
3       DD-Mon-YY  
4  Month DD, YYYY  
5           Other  

Age Group Distribution:
age_group
Unknown    6
Name: count, dtype: int64

Date Format Distribution:
date_format
DD-MM-YYYY        3
DD-Mon-YY         1
Month DD, YYYY    1
Other             1
Name: count, dtype: int64

Valid Dates:
is_valid_date
True     5
False    1
Nam